In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import sys
import os
import math

In [2]:
data_dir: os.PathLike = "/Volumes/Extreme Pro/datasets/NEMSIS/ASCII/"

nemsis_subset = {
    "ComputedElements":[
        'NasemsoRegion',
        'Urbanicity',
        'ageinyear',
        'EMSDispachcenterTimeSec',
        'EMSChuteTimeMin',
        'EMSSceneResponseTimeMin',
        'EMSSceneTimeMin',
        'EMSSceneToPatientTimeMin'
    ],
    "FACTPCRADDITIONALRESPONSEMODE":None,
    "FACTPCRADDITIONALSYMPTOM":None,
    "FACTPCRADDITIONALTRANSPORTMODE":None,
    "FACTPCRARRESTCPRPROVIDED":None,
    "FACTPCRARRESTRESUSCITATION":None,
    "FACTPCRARRESTRHYTHMDESTINATION":None,
    "FACTPCRARRESTROSC":None,
    "FACTPCRARRESTWITNESS":None,
    "FACTPCRBARRIERTOCARE":None,
    "FACTPCRMEDICATION":None,
    "FACTPCRPRIMARYIMPRESSION":None,
    "FACTPCRPROCEDURE":None,
    "FACTPCRPROTOCOL":None,
    "FACTPCRSECONDARYIMPRESSION":None,
    "FACTPCRTRAUMACRITERIA":None,
    "FACTPCRWORKRELATEDEXPOSURE":None,
    "PCRMEDCOMPGROUP":None,
    "PCRPROCCOMPGROUP":None,
    "PCRVITALECGGROUP":None,
    "Pub_PCRevents":[
        'eArrest_01',
        'eArrest_02',
        'eArrest_05',
        'eArrest_07',
        'eArrest_11',
        'eArrest_14',
        'eArrest_16',
        'eArrest_18',
        'eDisposition_23',
        'ePayment_01',
        'eResponse_07',
        'eResponse_15',
        'eSituation_01',
        'eSituation_07',
        'eSituation_08',
        'eTimes_01',
        'eTimes_03',
        'eTimes_05',
        'eTimes_06',
        'eTimes_07',
        'eTimes_09',
        'eTimes_11',
        'eTimes_12',
        'eTimes_13'
    ],
}


In [ ]:
def load_NEMSIS(data_dir: os.PathLike, load_targets: dict):
    
    data_dict = {}
    for f in load_targets.keys():
        data = dd.read_csv(os.path.join(data_dir, f + ".txt"), delimiter="~\|~", engine="python", blocksize=25e6)
        if load_targets[f] is not None:
            data = data[[load_targets[f]]]

        data_dict[f] = data

    return data_dict



data = load_NEMSIS(data_dir=data_dir, load_targets=nemsis_subset)


In [3]:
data = dd.read_csv(os.path.join(data_dir, 'ComputedElements' + ".txt"), delimiter="~\|~", engine="python", blocksize=64e6)

In [4]:
data.head()

,'PcrKey','CensusRegion','CensusDivision','NasemsoRegion','Urbanicity','ageinyear','EMSDispatchCenterTimeSec','EMSChuteTimeMin','EMSSystemResponseTimeMin','EMSSceneResponseTimeMin','EMSSceneTimeMin','EMSSceneToPatientTimeMin','EMSTransportTimeMin','EMSTotalCallTimeMin'
0,3442467,South,East South Central,South,,.,.,.,.,.,.,.,.,182
1,9059962,South,South Atlantic,South,Urban,.,.,.,.,.,.,.,.,24
2,9778729,West,Pacific,West,Rural,90,60,2,15,13,12,.,18,64
3,9829108,West,Pacific,West,Urban,85,10,4.8333333333,9.8333333333,5,60,.,5,84.833333333
4,10094816,South,South Atlantic,South,Urban,82,.,.,.,.,.,.,.,.


In [8]:
nemsis_subset['ComputedElements']

['NasemsoRegion',
 'Urbanicity',
 'ageinyear',
 'EMSDispachcenterTimeSec',
 'EMSChuteTimeMin',
 'EMSSceneResponseTimeMin',
 'EMSSceneTimeMin',
 'EMSSceneToPatientTimeMin']

In [ ]:
data = data[nemsis_subset['ComputedElements']]